In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from delta.tables import DeltaTable

spark = SparkSession.builder.appName("SalesAnalysis").getOrCreate()


In [0]:

# Create schema
spark.sql("CREATE SCHEMA IF NOT EXISTS finance.sales")


DataFrame[]

In [0]:
# Define schema
sales_schema = StructType([
    StructField("sale_id", IntegerType(), False),
    StructField("product", StringType(), True),
    StructField("amount", IntegerType(), True),
    StructField("quarter", StringType(), True)
])

# Initial sales data
initial_data = [
    (1, "Stocks", 10000, "Q1-2023"),
    (2, "Bonds", 15000, "Q1-2023")
]

In [0]:
initial_df = spark.createDataFrame(initial_data, schema=sales_schema)
initial_df.write.format("delta").mode("overwrite").saveAsTable("finance.sales.quarterly_sales")

In [0]:
# Update (correction)
update_data = [
    (1, "Stocks", 12000, "Q1-2023")
]
update_df = spark.createDataFrame(update_data, schema=sales_schema)

delta_table = DeltaTable.forName(spark, "finance.sales.quarterly_sales")
delta_table.alias("target").merge(
    update_df.alias("source"),
    "target.sale_id = source.sale_id"
).whenMatchedUpdate(set={
    "amount": "source.amount"
}).execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Get previous version number
history_df = delta_table.history()
previous_version = history_df.select("version").collect()[1][0]  # version before update

# Query previous version
prev_total = spark.sql(f"""
    SELECT SUM(amount) AS total_amount
    FROM finance.sales.quarterly_sales VERSION AS OF {previous_version}
""")
prev_total.show()

+------------+
|total_amount|
+------------+
|       27000|
+------------+



In [0]:

# Query current version
current_total = spark.sql("""
    SELECT SUM(amount) AS total_amount
    FROM finance.sales.quarterly_sales
""")
current_total.show()

+------------+
|total_amount|
+------------+
|       27000|
+------------+

